# Assignment 4 - TDT4265
#### Sara L. Ludvigsen and Emma H. Buøen
##### April 2020

## Task 1
### 1.a
The intersection over union (IoU), is the ratio between the intersected area over the joined area for two regions.
![IoU.png](IoU.png)

### 1.b
Precison measures how accurate your predictions is. Recall measures how good you find all the positives. When we say "true positive" we mean the cases where we predicted "positive" to be "true", and the prediction was correct. "False positive" is when we predict "positive", but it is incorrect. 

$$
TP = True Positive
$$
$$
TN = True Negative
$$
$$
FP = False Positive
$$
$$
FN = False Negative
$$

$$
Precision = \frac{TP}{TP + FP}
$$

$$
Recall = \frac{TP}{TP + FN}
$$

### 1.c

Average precision (AP) is equal to the area under the precision and recall curve. The mean average precision (mAP) is the mean of the curves. 

*Insert drawing of precision and recall curve*

$$
AP_{class1} = 0.4*1 + 0.3*0.5 + 0.3*0.2 + 0.3*0.5*0.5 + 0.3*0.3*0.5 = 0.73
$$

$$
AP_{class2} = 0.3*1 + 0.2*0.6 + 0.2*0.5 + 0.3*0.2 + 0.2*0.4*0.5 + 0.2*0.1*0.5 + 0.3*0.3*0.5 = 0.675
$$

$$
mAP = \frac{AP_{class1} + AP_{class2}}{2} = \frac{0.73 + 0.675}{2} = 0.7025
$$

## Task 2
### 2.a - 2.e 
was implemented in file `task2.py`.

### 2.f
![precision_recall_curve.png](task2/precision_recall_curve.png)

## Task 3
### 3.a
*The SSD architecture produces a fixed-size number of bounding boxes and a score for each bounding box. After this, the final step is to filter out overlapping boxes, what is this called?*

The final step to filter out overlapping boxes is called a non-maximum suppression step.

### 3.b
*The SSD architecture predicts bounding boxes at multiple scales to enable the network to detect objects of different sizes. Is the following true or false: Predictions from the deeper layers in SSD are responsible to detect small objects.*

The statement 

> Predictions from the deeper layers in SSD are responsible to detect small objects.

is false. The deeper layers are reponsible to detect larger objects. As stated in the [blog post](https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06) about SSDs 

> Higher resolution feature maps are responsible for detecting small objects

and the resoultion is highest at the beginning of the network, and is reduced for each layer.

### 3.c

*SSD use k number of bounding boxes with different aspect ratios at each spatial location in a feature map to predict c class scores and 4 offsets relative to the original box shape. Why do they use different bounding box aspect ratios at the same spatial location?Why do they use different bounding box aspect ratios at the same spatial location?*

They use diffrent aspect ratios for bounding boxes at the same spatial locations so they get a diverse set of predictions, because it covers various input object shapes and sizes. They also observed that more default boxes are better. When they use 6 boxes per location, the performance is 0.6% better than with 4 boxes. 4 boxes is 2.1% better than 2 boxes. It seems that using a variety of default box shapes makes the prediction easier for the network.

### 3.d
*What is the main difference between SSD and YOLOv1/v2 (The YOLO version they refer to in the SSD paper)?*

The main difference is that the SSD model adds several feature layers to the end of the network. These layers predict the offsets and confidences to default boxes of different scales and aspect ratios. Below is an image that illustrates the different network architectures.

![Graphics/task3d_comparision.png](Graphics/task3d_comparision.png)

The image is taken from [**SSD: Single Shot MultiBox Detector** by Wei Liu, Dragomir Anguelov, Dumitru Erhan, Christian Szegedy, Scott Reed, Cheng-Yang Fu, Alexander C. Berg](https://arxiv.org/abs/1512.02325).


## Task 4
### 4.a

The code was implemented in `basic.py`.
### 4.b
```
2020-04-14 21:35:58,315 SSD.trainer INFO: iter: 006000, lr: 0.00100, total_loss: 2.490 (3.411), reg_loss: 0.642 (0.846), cls_loss: 1.848 (2.566), time: 0.099 (0.097), eta: 3:03:26, mem: 2075M
2020-04-14 21:35:58,343 SSD.trainer INFO: Saving checkpoint to outputs/basic/model_006000.pth
2020-04-14 21:35:58,622 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 35.92it/s]
2020-04-14 21:36:02,232 SSD.inference INFO: mAP: 0.7746
```

After 6000 iterations, our mean Average Precision was **77.46%**. 

Below is a plot of the total loss, taken from Tensorboard.

![Graphics/task4b.png](Graphics/task4b.png)

### 4.c

When changing the optimizer to Adam, and the learning rate to $2e-3$ and adding a BatchNorm2d layer to every feature bank, we achieved a mAP og **85.9%** after 9500 iterations!

```
2020-04-15 15:32:15,736 SSD.trainer INFO: iter: 009500, lr: 0.00020, total_loss: 1.696 (2.280), reg_loss: 0.434 (0.593), cls_loss: 1.262 (1.687), time: 0.099 (0.100), eta: 0:04:09, mem: 2075M
2020-04-15 15:32:15,757 SSD.trainer INFO: Saving checkpoint to outputs/basic/model_009500.pth
2020-04-15 15:32:16,145 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 34.23it/s]
2020-04-15 15:32:19,983 SSD.inference INFO: mAP: 0.8589
0               : 0.8779
1               : 0.7953
2               : 0.8640
3               : 0.8828
4               : 0.8665
5               : 0.8716
6               : 0.8695
7               : 0.8428
8               : 0.8803
9               : 0.8379
```

This is the new and improved network structure:

```python
self.bank1 = nn.Sequential(
            nn.Conv2d(
                in_channels=image_channels,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            ),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            ),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=64,
                out_channels=output_channels[0],
                kernel_size=3,
                stride=2,
                padding=1
            )
        )
        self.bank2 = nn.Sequential(
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=output_channels[0],
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=128,
                out_channels=output_channels[1],
                kernel_size=3,
                stride=2,
                padding=1
            )
        )
        self.bank3 = nn.Sequential(
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=output_channels[1],
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(256),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=256,
                out_channels=output_channels[2],
                kernel_size=3,
                stride=2,
                padding=1
            )
        )
        self.bank4 = nn.Sequential(
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=output_channels[2],
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=128,
                out_channels=output_channels[3],
                kernel_size=3,
                stride=2,
                padding=1
            )
        )
        self.bank5 = nn.Sequential(
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=output_channels[3],
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=128,
                out_channels=output_channels[4],
                kernel_size=3,
                stride=2,
                padding=1
            )
        )
        self.bank6 = nn.Sequential(
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=output_channels[4],
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=1
            ), 
            nn.BatchNorm2d(128),
            nn.ReLU(), 
            nn.Conv2d(
                in_channels=128,
                out_channels=output_channels[5],
                kernel_size=3,
                stride=1,
                padding=0
            )
        )
        self.feature_extractor = nn.Sequential(
            self.bank1,
            self.bank2,
            self.bank3,
            self.bank4,
            self.bank5,
            self.bank6
        )
 ```

### 4.d

In this task we changed the `MIN_SIZES` and `MAX_SIZES` for the model. We added the following to the `mnist.yaml` file:
```python
MIN_SIZES: [21, 45, 99, 153, 207, 261]
MAX_SIZES: [45, 99, 153, 207, 261, 315]
```
```
2020-04-15 20:48:43,930 SSD.trainer INFO: iter: 012500, lr: 0.00020, total_loss: 1.204 (1.731), reg_loss: 0.332 (0.438), cls_loss: 0.872 (1.293), time: 0.080 (0.097), eta: 0:07:17, mem: 2075M
2020-04-15 20:48:43,961 SSD.trainer INFO: Saving checkpoint to outputs/basic/model_012500.pth
2020-04-15 20:48:46,727 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 34.40it/s]
2020-04-15 20:48:50,429 SSD.inference INFO: mAP: 0.9020
0               : 0.9078
1               : 0.8692
2               : 0.9063
3               : 0.9075
4               : 0.9076
5               : 0.9042
6               : 0.9065
7               : 0.9025
8               : 0.9064
9               : 0.9017
```

Iterations | mAP
---|---
12500|90.20%
13000|90.02%
13500|90.04%
14000|90.12%
14500|90.06%
15000|90.20%

We achieved these results by changing the config file `mnist.yaml` to the following:
```python
MODEL:
    NUM_CLASSES: 11
    BACKBONE:
        NAME: 'basic'
        PRETRAINED: False
        OUT_CHANNELS: [128, 256, 128, 128, 64, 64]
        INPUT_CHANNELS: 3
    PRIORS:
        FEATURE_MAPS: [38, 19, 10, 5, 3, 1]
        STRIDES: [8, 16, 32, 64, 100, 300]
        MIN_SIZES: [21, 45, 99, 153, 207, 261]
        MAX_SIZES: [45, 99, 153, 207, 261, 315]
        ASPECT_RATIOS: [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
        BOXES_PER_LOCATION: [4, 6, 6, 6, 4, 4]
INPUT:
    IMAGE_SIZE: 300
DATASETS:
    TRAIN: ("mnist_detection_train", "mnist_detection_val")
    TEST: ("mnist_detection_val", )
SOLVER:
    MAX_ITER: 17000
    LR_STEPS: [80000, 100000]
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 2e-4
OUTPUT_DIR: 'outputs/basic'
DATASET_DIR: "datasets"
```

### 4.e

Here are the images from the demo-file:


Image 0,3,6,9,12|Image 1,4,7,10,13|Image 2,5,8,11,14
:--------------:|:---------------:|:----------------:
![](Graphics/demo/download-0.png)  |  ![](Graphics/demo/download-1.png) |![](Graphics/demo/download-2.png) 
![](Graphics/demo/download-3.png)  |  ![](Graphics/demo/download-4.png) |![](Graphics/demo/download-5.png) 
![](Graphics/demo/download-6.png)  |  ![](Graphics/demo/download-7.png) |![](Graphics/demo/download-8.png) 
![](Graphics/demo/download-9.png)  |  ![](Graphics/demo/download-10.png) |![](Graphics/demo/download-11.png) 
![](Graphics/demo/download-12.png)  |  ![](Graphics/demo/download-13.png) |![](Graphics/demo/download-14.png) 

We observe that our model struggles the most with small-sized numbers and the number 1. Sometimes it also misses overlapping numbers.

### 4.f

```
2020-04-15 23:06:37,770 SSD.trainer INFO: iter: 005000, lr: 0.00050, total_loss: 4.660 (5.513), reg_loss: 1.320 (1.660), cls_loss: 3.340 (3.854), time: 0.799 (0.860), eta: 10:45:04, mem: 11167M
2020-04-15 23:06:38,069 SSD.trainer INFO: Saving checkpoint to outputs/vgg_VOC/model_005000.pth
2020-04-15 23:06:38,378 SSD.inference INFO: Evaluating voc_2007_test dataset(4952 images):
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 496/496 [03:23<00:00,  3.09it/s]
2020-04-15 23:10:05,051 SSD.inference INFO: mAP: 0.3288
aeroplane       : 0.4453
bicycle         : 0.3762
bird            : 0.1836
boat            : 0.1761
bottle          : 0.0914
bus             : 0.3907
car             : 0.5665
cat             : 0.4115
chair           : 0.1784
cow             : 0.3674
diningtable     : 0.2833
dog             : 0.2655
horse           : 0.4536
motorbike       : 0.3886
person          : 0.5356
pottedplant     : 0.0878
sheep           : 0.2822
sofa            : 0.2841
train           : 0.3794
tvmonitor       : 0.4291
```

![](Graphics/demo/download-15.png)  
![](Graphics/demo/download-16.png) 
![](Graphics/demo/download-17.png) 
![](Graphics/demo/download-18.png)  
![](Graphics/demo/download.png) 

![](Graphics/demo/total_plot.png)
